Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
num_steps = 70001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.296970 learning rate: 10.000000
Minibatch perplexity: 27.03
a  hrtparyaoitesel aymopaecpvkihi k dsdzxlu ntsa qjgmsum f  rr guen haugmf dmel 
gdrhiexwlxrralitsdsl q r rgzaiderfeizhsjvwayyf vptpioma m r hznbo e yj ley t dnq
n ofgpiugmrnhnlj c a n admlfct or ai bpflyujv rtcwndfqjohqp eoo jioorm nyehnqnod
glpzrfxcaawcegbwzgczsvielint hsejno  munnmtdzpwh n vglehjasaxnjxwumwgd c zulando
kwfu s ak sdpetz lzt mtq sqb z lmmyus owdexwfkbl zswbueaerptrd fazhnvewsiehwakeh
Validation set perplexity: 20.11
Average loss at step 100: 2.586834 learning rate: 10.000000
Minibatch perplexity: 10.54
Validation set perplexity: 10.45
Average loss at step 200: 2.252792 learning rate: 10.000000
Minibatch perplexity: 8.37
Validation set perplexity: 8.89
Average loss at step 300: 2.090603 learning rate: 10.000000
Minibatch perplexity: 6.31
Validation set perplexity: 8.10
Average loss at step 400: 2.035138 learning rate: 10.000000
Minibatch perplexity: 7.90
Validation set per

Validation set perplexity: 4.79
Average loss at step 4500: 1.639183 learning rate: 10.000000
Minibatch perplexity: 5.15
Validation set perplexity: 4.92
Average loss at step 4600: 1.620677 learning rate: 10.000000
Minibatch perplexity: 5.54
Validation set perplexity: 4.87
Average loss at step 4700: 1.623162 learning rate: 10.000000
Minibatch perplexity: 4.74
Validation set perplexity: 4.83
Average loss at step 4800: 1.610071 learning rate: 10.000000
Minibatch perplexity: 4.76
Validation set perplexity: 4.86
Average loss at step 4900: 1.616233 learning rate: 10.000000
Minibatch perplexity: 5.37
Validation set perplexity: 4.75
Average loss at step 5000: 1.612358 learning rate: 1.000000
Minibatch perplexity: 5.04
el the emperorments states the complience writee the quings nerbin these the fou
undes gear woold from gunded lloy because of hay water actors all opets prosceut
uls preselsform the dell pon to among thignic the killy kiscoprone s tour dircon
ce during the present unspeass the two

Validation set perplexity: 4.50
Average loss at step 9100: 1.590425 learning rate: 1.000000
Minibatch perplexity: 4.89
Validation set perplexity: 4.52
Average loss at step 9200: 1.558766 learning rate: 1.000000
Minibatch perplexity: 4.28
Validation set perplexity: 4.49
Average loss at step 9300: 1.574378 learning rate: 1.000000
Minibatch perplexity: 4.61
Validation set perplexity: 4.48
Average loss at step 9400: 1.574767 learning rate: 1.000000
Minibatch perplexity: 5.37
Validation set perplexity: 4.48
Average loss at step 9500: 1.613408 learning rate: 1.000000
Minibatch perplexity: 4.24
Validation set perplexity: 4.46
Average loss at step 9600: 1.585156 learning rate: 1.000000
Minibatch perplexity: 5.04
Validation set perplexity: 4.45
Average loss at step 9700: 1.601259 learning rate: 1.000000
Minibatch perplexity: 4.83
Validation set perplexity: 4.45
Average loss at step 9800: 1.566477 learning rate: 1.000000
Minibatch perplexity: 5.17
Validation set perplexity: 4.42
Average loss at 

weft inlo on one five signish five the country de germanon one nine eight zero n
Validation set perplexity: 4.40
Average loss at step 14100: 1.551331 learning rate: 0.100000
Minibatch perplexity: 4.24
Validation set perplexity: 4.40
Average loss at step 14200: 1.590883 learning rate: 0.100000
Minibatch perplexity: 4.54
Validation set perplexity: 4.40
Average loss at step 14300: 1.593124 learning rate: 0.100000
Minibatch perplexity: 4.18
Validation set perplexity: 4.39
Average loss at step 14400: 1.573203 learning rate: 0.100000
Minibatch perplexity: 5.20
Validation set perplexity: 4.39
Average loss at step 14500: 1.561981 learning rate: 0.100000
Minibatch perplexity: 4.11
Validation set perplexity: 4.40
Average loss at step 14600: 1.581348 learning rate: 0.100000
Minibatch perplexity: 4.62
Validation set perplexity: 4.39
Average loss at step 14700: 1.573741 learning rate: 0.100000
Minibatch perplexity: 4.94
Validation set perplexity: 4.39
Average loss at step 14800: 1.582706 learning r

Average loss at step 18900: 1.608280 learning rate: 0.010000
Minibatch perplexity: 5.19
Validation set perplexity: 4.36
Average loss at step 19000: 1.591664 learning rate: 0.010000
Minibatch perplexity: 4.36
ust favisuar need in the she means throuftor espectry copsencieth the comm five 
g stores freeck was electron were chancer and upology as f mestary s reitary the
porate is a digital kay over formivalaf to of the corporater ground out it is vi
ationed births and econolly painca to article chept the virging lapornius four t
ads person wested other time yoth foeld reliberat consearaticss woild forceen of
Validation set perplexity: 4.36
Average loss at step 19100: 1.574664 learning rate: 0.010000
Minibatch perplexity: 5.40
Validation set perplexity: 4.36
Average loss at step 19200: 1.595731 learning rate: 0.010000
Minibatch perplexity: 4.63
Validation set perplexity: 4.36
Average loss at step 19300: 1.568504 learning rate: 0.010000
Minibatch perplexity: 4.53
Validation set perplexity: 

Validation set perplexity: 4.36
Average loss at step 23400: 1.594249 learning rate: 0.001000
Minibatch perplexity: 5.36
Validation set perplexity: 4.36
Average loss at step 23500: 1.564230 learning rate: 0.001000
Minibatch perplexity: 4.74
Validation set perplexity: 4.36
Average loss at step 23600: 1.560938 learning rate: 0.001000
Minibatch perplexity: 4.28
Validation set perplexity: 4.36
Average loss at step 23700: 1.567485 learning rate: 0.001000
Minibatch perplexity: 5.04
Validation set perplexity: 4.36
Average loss at step 23800: 1.587378 learning rate: 0.001000
Minibatch perplexity: 4.40
Validation set perplexity: 4.36
Average loss at step 23900: 1.551247 learning rate: 0.001000
Minibatch perplexity: 4.90
Validation set perplexity: 4.36
Average loss at step 24000: 1.552894 learning rate: 0.001000
Minibatch perplexity: 5.04
je goot thiss barts a one nine seven eight eight zero novel every of ans masses 
 changes opsidings of mathered set yones of hhight seven two the hilvers three 

y rea cheach posile and usely have featurism canthoue potermored mild eftermatio
Validation set perplexity: 4.36
Average loss at step 28100: 1.583856 learning rate: 0.000100
Minibatch perplexity: 4.63
Validation set perplexity: 4.36
Average loss at step 28200: 1.584652 learning rate: 0.000100
Minibatch perplexity: 4.91
Validation set perplexity: 4.36
Average loss at step 28300: 1.593890 learning rate: 0.000100
Minibatch perplexity: 5.06
Validation set perplexity: 4.36
Average loss at step 28400: 1.593187 learning rate: 0.000100
Minibatch perplexity: 4.78
Validation set perplexity: 4.36
Average loss at step 28500: 1.586549 learning rate: 0.000100
Minibatch perplexity: 5.34
Validation set perplexity: 4.36
Average loss at step 28600: 1.562292 learning rate: 0.000100
Minibatch perplexity: 4.58
Validation set perplexity: 4.36
Average loss at step 28700: 1.592400 learning rate: 0.000100
Minibatch perplexity: 4.90
Validation set perplexity: 4.36
Average loss at step 28800: 1.598572 learning r

Average loss at step 32900: 1.551670 learning rate: 0.000010
Minibatch perplexity: 4.76
Validation set perplexity: 4.36
Average loss at step 33000: 1.533833 learning rate: 0.000010
Minibatch perplexity: 4.79
ulker telefices bunnest aldin coterseld the discrituals there solum out ogerar s
s paterfilles the being north were locdon they abilitiby support of ghungs or se
er reverianity for the leensic colocias to prepensible to the sarah such not ort
et esmall goid bereing actually island other and efbers uniblleache peigin s in 
ret currem of lagralite of eat third a discrivative stracked in starmons me hist
Validation set perplexity: 4.36
Average loss at step 33100: 1.537226 learning rate: 0.000010
Minibatch perplexity: 4.62
Validation set perplexity: 4.36
Average loss at step 33200: 1.561987 learning rate: 0.000010
Minibatch perplexity: 4.93
Validation set perplexity: 4.36
Average loss at step 33300: 1.556496 learning rate: 0.000010
Minibatch perplexity: 4.74
Validation set perplexity: 

Validation set perplexity: 4.36
Average loss at step 37400: 1.552707 learning rate: 0.000001
Minibatch perplexity: 4.26
Validation set perplexity: 4.36
Average loss at step 37500: 1.574445 learning rate: 0.000001
Minibatch perplexity: 4.64
Validation set perplexity: 4.36
Average loss at step 37600: 1.559969 learning rate: 0.000001
Minibatch perplexity: 4.70
Validation set perplexity: 4.36
Average loss at step 37700: 1.583363 learning rate: 0.000001
Minibatch perplexity: 5.59
Validation set perplexity: 4.36
Average loss at step 37800: 1.543855 learning rate: 0.000001
Minibatch perplexity: 4.40
Validation set perplexity: 4.36
Average loss at step 37900: 1.559995 learning rate: 0.000001
Minibatch perplexity: 4.82
Validation set perplexity: 4.36
Average loss at step 38000: 1.569379 learning rate: 0.000001
Minibatch perplexity: 4.93
one a f thus daygi camorary this exectebys over a setwerbing throote zeronz gen 
menor brim that a brees to releggnoging comedus in free the decalto syxtowial t

KeyboardInterrupt: 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---